In [96]:
import os
import json
import re
import pandas as pd

In [97]:
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [98]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [99]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [100]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [101]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )


In [102]:
TEMPLATE2="""
You are an expert english grammarian and writer.
If the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}
(ONLY generate the updated quiz in the quiz format, nothing more)
Check from an expert English Writer of the above quiz:
"""

In [103]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["quiz"], template=TEMPLATE2)

In [104]:
TEMPLATE3='''
{updated_quiz}
Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question but ONLY use at max 50 words for complexity analysis. DON'T Mention the quiz is revised or updated
'''

In [105]:
complexity_analysis_prompt=PromptTemplate(input_variables=["updated_quiz", "subject"], template=TEMPLATE3)

In [106]:
with open(r'C:\Users\afz31\mcq_gen\data.txt') as file:
        TEXT = file.read()

In [107]:
NUMBER=5
SUBJECT="biology"
TONE="simple"

In [108]:
def generate_quiz():
    model_options = ["gemini-1.5-pro", "gemini-1.5-flash-8b"]
    for model in model_options:
        try:
            llm = ChatGoogleGenerativeAI(model=model)
            generate_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)
            evaluate_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="updated_quiz", verbose=True)
            analysis_chain = LLMChain(llm=llm, prompt=complexity_analysis_prompt, output_key="complexity_analysis", verbose=True)
            
            gen_eval_analysis_chain = SequentialChain(
                chains=[generate_chain, evaluate_chain, analysis_chain],
                input_variables=["text", "number", "subject", "tone", "response_json"],
                output_variables=["updated_quiz", "complexity_analysis"],
                verbose=True,
            )
            
            return gen_eval_analysis_chain({
                "text": TEXT,
                "number": NUMBER,
                "subject": SUBJECT,
                "tone": TONE,
                "response_json": json.dumps(RESPONSE_JSON),
            })
        except Exception as e:
            print(f"Error with model {model}: {e}")
            print("Trying again with the next available model...")

    raise RuntimeError("All models failed to generate the quiz.")

In [109]:
response = generate_quiz()



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The PyPDF2 package is a pure-Python PDF library that you can use for splitting, merging, cropping and transforming pages in your PDFs. It can also add custom data, viewing options, and passwords to PDF files. It can extract text data from PDF files, and you can also use it to read and modify existing PDF files.

You are an expert MCQ maker. Given the above text, it is your job to create a quiz  of 5 multiple choice questions for biology students in simple tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make 5 MCQs
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice que

In [110]:
# Regular expression to extract MCQs, options, and answers
pattern = r'"mcq": "(.*?)".*?"options": \{(.*?)\}.*?"correct": "(.*?)"'

# Find all matches
matches = re.findall(pattern, response.get("updated_quiz"), re.DOTALL)

# Process each match
quiz_data = []
for mcq, options_str, correct in matches:
    # Extract individual options
    options_pattern = r'"([a-d])": "(.*?)"'
    options = re.findall(options_pattern, options_str)

    # Create a dictionary for the question
    question = {
        "mcq": mcq.strip(),
        "options": {key: value.strip() for key, value in options},
        "correct": correct.strip(),
    }
    quiz_data.append(question)

In [111]:
quiz=pd.DataFrame(quiz_data)

In [112]:
quiz.to_csv("quiz_data.csv",index=False)